# **Swarm Intelligence**
Swarm intelligence is an artificial intelligence techinique based on the study of collective behaviour in decentralized self-organized systems.
These systems are typically populations of simple agents interacting locally with each other and the environment.
Even though there is **no centralized control**, these local interactions often lead to the **emergence of a global behaviour**.

Human intelligence is the result of social interaction as well: evaluate, compare, learning from experince emulating succesfull behaviours,...

Culture and cognition are consequences of human social choices.
In fact, culture emerges when individual become similar through social learning.
So, in order to model human intelligence, there's need to model individuals in a social context.

#Features of a Swarm Intelligence system
* Set of **simple individuals** that aren't aware of the system in global view;
* Local communication either direct or indirect (**stigmergic**);
* Distribuited computation (not centralized);
* Robustness, achieved through **graceful degradation**;
* Adaptivity.

**Stigmergy**: a form of indirect communication, when an agent modifies the environment other agents may recoil and react to this.

**Graceful adaptation**: when part of the system is destroyed, it still mantains (partial) functionality.

**Self-organization**: is based on:
* **Multiple interactions** among agents, in a probabilistic way. Can be simple agents or multi-agents systems;
* **Positive feedback**: reinforcement of good common behaviour and amplification of random fluctuations and structure formation;
* **Negative feedback**: saturation, competition and resources exhaustation.

Many different algorithms exist:
* **Ant colony optimization** (ACO): positive feedback given by pheromone trails that reinforce components that contribute to the problem's solution;
* **Artificial bee colony** (ABC): individuals with different functions;
* **Particle swarm optimization** (PSO): stigmergy as communication and imitation of neighbourhoods.

# Ant colony optimization (ACO)
From ants observation we discover that:
* Ants release **pheromones** trails while walking from the nest to the food and viceversa;
* Ants tend to chose more likely the paths marked with higher pheromone levels (but not always, is not deterministic);
* Cooperative interaction leads to an emergent behaviour to find the shortest path.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/Aco_branches.svg/1280px-Aco_branches.svg.png)

If an ant finds food it follows the pheromones left along the path to reach it and take the food back to the nest. Shorter paths will be reinforced by pheromones further, while in longer paths pheromones will evaporate.

The **ant colony optimization** is a *probabilistic parametrized model* where ants (agents) build solution components in an incremental way.
The algorithm takes stochastic steps on a fully connected graph called *construction graph* $G=(C,L)$, where:
* Vertexes $C$ are solution components;
* Arcs $L$ are connections;
* States are paths on $G$.

Constraints may be represented to define what is a consistent solution.

Example of an implementation of a TSP model in ACO: nodes of the graph $G$ are the solution to be visited (solution components), arcs are connections between cities and a solution is an Hamiltonian path in the graph. Constraint are used to avoid sub-cycles (each ant can visit a city once).

# The algorithm
Connection, solution components or both have associated:
* Pheromone $\tau$, that abstracts natural pheromone trails and codes long term memory of the global search process;
* Heuristic value $\eta$, that represents the prior background knowledge on the problem.

Pheromones take into account all agents and each agent also uses heuristics to search locally.
![](https://i.ibb.co/x5rjQCV/schema.png)

Ants build a solution following a path on the *construction graph* $G$, a transition rule is followed to choose the next node to visit. Pheromones values are updated on the basis of the **quality** of the solution found.

> **Ant-system algorithm**:
* Initialize pheromone values (if 0 it would starts go around randomly);
* While termination conditions aren't met, for all ants $a \in A$ construct a solution $s_a$ based on $\tau$ and $\eta$;
* Update the pheromones (**delayed update**: we update the pheromone if we find the solution);
* Repeat from step 2.

Memory is used to remind past paths. Starting from node $i$, we have to probabilistically choose the next consistent node, and that choice depends, as we said before, on:
* Pheromone trail $\tau_{ij}$ (global);
* Heuristics $\eta_{ij} = \frac{1}{d_{ij}}$ (individual).

The probability will be:
* $p_{ij} = \frac {[{\tau_{ij}}]^\alpha[{\eta_{ij}}]^\beta}{\sum_{k \text{ feasible}}[{\tau_{ik}}]^\alpha[{\eta_{ik}}]^\beta}$ if $j$ is consistent (so I can go there with my problem);
* $p_{ij} = 0$ otherwise.

Where $\alpha$ and $\beta$ weight the importance of the pheromone and the heuristics.


The pheromone is updated with the following rule: $\tau_{ij} \leftarrow (1-\rho)\tau_{ij} + \sum_{k=1}^{m} \Delta \tau_{ij}^{k}$, where:
* $\rho$ is the evaporation coefficient;
* $\Delta \tau_{ij}^{k} = \frac {1}{L_{k}}$ if ant $k$ used the arc $(i,j)$ and $0$ otherwise;
* $L_{k}$ is the lenght of the path followed by ant $k$.

The high level algorithm is:
> * While termination condition aren't met:
* `AntBasedSolutionConstruction()`;
* `PheromoneUpdate()`;
* `DaemonActions()`; 
* End while.

`AntBasedSolutionConstruction()`: ants move by applying a stochastic local decision policy (seen before). While moving, ants take track of the partial solutions (paths) that have been built.

`PheromoneUpdate()`: can be of two kind:
* Online step-by-step pheromone update: ants update during the solution construction;
* Online delayed pheromone update: ants update backward on the basis of the quality of the overall solution.

In any case evaporation is applied all the time.

`DaemonActions()`: centralized actions to boost the algorithm (it's like cheating). Local search can be applied to each built solution to improve it.



# Artificial Bee Colony (ABC)
The bees are of three types:
* **Employed bees**: associated with a specific nectar source;
* **Onlooker bees**: observe the employed bees and choose a nectar source;
* **Scout bees**: discover new food sources.

Initially, food sources are discovered by scout bees. When food is consumed and the source exhausted the employed bees in that source become scouts.

**Food position** | **Food quantity**
--- | ---
Solution | Fitness
*Exploration* | *Exploitation*

We have as many solutions as the employed bees.

> **ABC algorithm**:
* `InitializationPhase()`;
* **Repeat**
* `EmployedBeePhase()`;
* `OnlookerBeePhase()`;
* `ScoutBeePhase()`;
* Solution $\leftarrow$ best solution so far;
* **Until** a termination condition is met.

`InitializationPhase()`: a set of food source positions are randomly selected by the bees and their nectar amount are determined. Each solution $x_m$ is composed of $n$ variables $x_{m_i}$. Each variable is subjected to a lower and upper bound and it's initialized to $x_{m_i} = \text{LB}_i + \text{rand}(0,1)\bigl[\text{UB}_i - \text{LB}_i\bigr]$.

`EmployedBeePhase()`: after sharing the information about the nectar amount, every employed bee goes to the food source visited by herself at the previous cycle (since it exist in it's memory) and then chooses a new food source by means of visual information in the **neighbourhood** (local search). We use an objective function to compute fitness as follows:
* $\text{ftn}(x_m)=\frac{1}{1+\text{obj}(x_m)}$ if $\text{obj}(x_m)\ge 0$
* $\text{ftn}(x_m)=1+ \lvert \text{obj}(x_m)\rvert$ if $\text{obj}(x_m)< 0$

`OnlookerBeePhase()`: onlooker bees choose a food source depending on the probability associated with that source: $p_m=\frac{\text{ftn}(x_m)}{\sum_{i=1}^{N}\text{ftn}(x_i)}$, therefore the ratio between the fitness of that specific solution and the sum of the fitness of all solutions. This is a **positive feedback**.

`ScoutBeePhase()`: scout bees choose nectar sources randomly (a kind of restart, diversification process). Employed bees that can't improve the solution after a given number of attempts become scout and leave the food source. This is a **negative feedback**.

Some applications of this algorithm are: training of neural networks, numerical optimization and combinatorial optimization.


# Particle Swarm Optimization (PSO)
This algorithm it's based on the analysis of interaction mechanism between individual that compose a swarm.
The observation of rules that guide a bird flock show that each individual tends to:
* Follow neighbours;
* Stay in the flock;
* Avoid collisions.

With these rules the model of a flock has no common objective.
PSO adds one: food search.
With a common objective each individual tends to:
* Move away from the group to reach the food (individualistic choice);
* Stay in the group (social choice).

If more than one individual entity moves toward the food, other members will do the same. Gradually the whole flock changes direction toward the promising area: the **information propagates** to all members.

With PSO it's possible to solve optimization problems with the following analogies:
 
* **Individuals**: tentative configuration that move and sample the solution (*exploration*);
* **Social interaction**: each individual agent takes advantage from other searches moving towards promising regions (best solution globally found, *exploitation*).

Search strategy can be found as a balance between *exploration* and *exploitation*.

**Neighbourhood**: in this algorithm it's important the concept of proximity.
Individuals are affected by the actions of other individuals that are closer to them (sub-grouping), imitating the most performant.
Individuals are parts of more sub-groups and in this way the **information spreads globally**.
Sub-groups aren't tied to the physical proximity of the configurations in the *parameter space*, but are apriori defined and so they may also take into account considerable shifts between individuals.

# The algorithm
PSO optimizes a problem by settig a population (the swarm) of candidate solutions (the particles).
PSO moves the particles in the search space through simple mathematical formulas: the movement is guided by the best position found so far in search space and it's updated when better solutions are discovered.
* **Fitness function** to minimize $f \colon \mathbb{R}^n \to \mathbb{R}$: it takes a solution ($n$-dimensional vector) and produces a fitness value. The gradient of $f$ is unknown;
* **Goal**: find a solution $s$ such that $f(s) \le f(s')$ for all $s'$ in the search space.

There are some parameters too:
* $S$ is the number of particles in population. Each particle has: a position $x_i \in \mathbb{R}^n$ in the search space and a velocity $v_i \in \mathbb{R}^n$;
* $p_i$ is the best solution found so far by particle $i$;
* $g$ is the best solution found so far by the entire swarm.

For each particle $i=1, \dots, S$ do (initialization phase):
1. Initialize the particle's position with an uniformly distribuited random vector $x_i \leftarrow \text{U}(b_{\text{low}},b_{\text{up}})$;
1. Initialize the particle's best know position to its initial position $p_i \leftarrow x_i$;
1. If $f(p_i) \le f(g)$ (means that $p_i$ is a better solution than $g$) update the swarm's best known position $g \leftarrow p_i$;
1. Initialize the particle's velocity with an uniformly distribuited random vector $v_i \leftarrow \text{U}( -\lvert b_{\text{up}}-b_{\text{low}}\rvert,\lvert b_{\text{up}}-b_{\text{low}}\rvert)$;

Then, until a termination criterion is met, for each particle $i=1, \dots, S$ do (swarm moving):
* Pick random numbers $r_p,r_g=\text{U}(0,1)$;
* Update the particle's velocity $v_i \leftarrow \omega v_i + \varphi_p r_p (p_i - x_i) + \varphi_g r_g (g-x_i)$;
* Update the particle's position $x_i \leftarrow x_i + v_i$;
* If $f(x_i)< f(p_i)$ (means that the new solution is better than the previous), then:
> * Update the particle's best know position $p_i \leftarrow x_i$;
  * Then if $f(p_i)< f(g)$ (means that the new solution is even better than the global one) update the swarm's best known position  $g \leftarrow p_i$;

At the end return the best solution found $g$.

Parameters $\omega$, $\varphi_p$ and $\varphi_g$ should be carefully selected as they strongly influence the effectiveness and efficiency of the method.
It's often used an automatic tuning done through iterated local search in the space of the parameters (as a proxy).





